# RogueSky

author: camen  
last run: 2019-12-12

In [1]:
import contextlib
import json
import logging
import os

import requests_mock

from rogue_sky import darksky, stars

DARKSKY_API_KEY = "test_api_key"

COORDINATES = (47.6062, -122.3321)  # seattle
DARKSKY_URL = f"https://api.darksky.net/forecast/{DARKSKY_API_KEY}/{COORDINATES[0]},{COORDINATES[1]}"

logging.basicConfig(level=logging.INFO)

## Table of Contents

1. [Set Up](#Set-Up)
2. [Introduction](#Introduction)
3. [Working with DarkSky](#Working-with-DarkSky)
    1. [Using the main entry point](#Using-the-weather-main-entry-point...)
    2. [Going through the steps](#Going-through-the-steps...)
        2. [From DarkSky](#From-DarkSky)
        4. [Serialize the output](#Serialize-the-output)
    3. [Understanding the data](#Understanding-the-data)
4. [Star Predictions](#Star-Predictions)
    1. [Using the main entry point](#Using-the-stars-main-entry-point...)

## Set Up

In [2]:
# set up the mock for requests (we won't actually hit the DarkSky API)
@contextlib.contextmanager
def mock_darksky_api():
    with requests_mock.Mocker() as mock:
        with open("resources/darksky_response.json") as json_file:
            data = json.load(json_file)
        mock.get(DARKSKY_URL, json=data)
        yield

## Introduction

Predict star visibility from DarkSky's daily weather forecast. In this notebook, we will query DarkSky for a daily weather forecast, show some information about that forecast, and predict the star visibility from it.

## Working with DarkSky

### Using the weather main entry point...

First, lets see it all work together... The main entry point into the darksky module is the `get_weather_forecast` method. This method:
    1. Hits the DarkSky API for the daily weather forecast
    3. Serializes the daily weather forecast into a JSON-parseable response that can be sent from the RogueSky API to the RogueSky frontend

The final output is of this form: 
```
{
    "0": {
        "latitude": 42.3601,
        "longitude": -71.0589,
        "queried_date_utc": "2019-01-01",
        "weather_date_local": "2019-11-10",
        "weather_json": weather_json,
    },
    "1": {
        "latitude": 42.3601,
        "longitude": -71.0589,
        "queried_date_utc": "2019-01-01",
        "weather_date_local": "2019-11-11",
        "weather_json": weather_json,
    },
    ...
}
```

In [3]:
with mock_darksky_api():
    backend_weather_response = darksky.get_weather_forecast(
        latitude=COORDINATES[0], 
        longitude=COORDINATES[1],
        api_key=DARKSKY_API_KEY
    )

print(backend_weather_response.keys())
backend_weather_response["daily_forecast"][0]

INFO:rogue_sky.darksky:Getting daily weather forecast for (47.6062, -122.3321) on 2019-12-12
INFO:rogue_sky.darksky:(47.6062, -122.3321, 2019-12-12): Requesting from DarkSky with parameters {'query_parameters': {'exclude': ['alerts', 'flags', 'hourly', 'minutely', 'offset', 'currently']}}
INFO:rogue_sky.darksky:(47.6062, -122.3321, 2019-12-12): Serializing to API output...


dict_keys(['latitude', 'longitude', 'queried_date_utc', 'daily_forecast'])


{'cloud_cover_pct': 0.93,
 'dew_point_f': 48.57,
 'humidity_pct': 0.94,
 'icon': 'fog',
 'moon_phase_pct': 0.46,
 'ozone': 262.2,
 'precip_intensity_avg_in_hr': 0.0004,
 'precip_intensity_max_in_hr': 0.0024,
 'precip_probability': 0.34,
 'precip_type': 'rain',
 'pressure': 1026.1,
 'summary': 'Foggy in the morning.',
 'sunrise_time_local': '2019-11-10T07:09:00-08:00',
 'sunset_time_local': '2019-11-10T16:41:00-08:00',
 'temperature_max_f': 53.25,
 'temperature_min_f': 44.73,
 'uv_index': 1,
 'visibility_mi': 5.174,
 'weather_date_local': '2019-11-10',
 'wind_gust_mph': 11.25,
 'wind_speed_mph': 3.93}

### Going through the steps...

1. Request from darksky (`darksky._from_darksky`)
3. Serialize (`darksky._serialize`) and return

#### From DarkSky

```
[
    {
        latitude: 42.3601,
        longitude: -71.0589,
        queried_date_utc: "2019-01-01",
        weather_date_local: "2019-01-01",
        weather_json: JSON({
            ...
        })
    },
    ...
]
```

In [4]:
# Query DarkSky and parse...
with mock_darksky_api():
    darksky_weather_response = darksky._from_darksky(
        latitude=COORDINATES[0],
        longitude=COORDINATES[1],
        queried_date_utc=backend_weather_response["queried_date_utc"],
        api_key=DARKSKY_API_KEY,
    )

darksky_weather_response[0]

INFO:rogue_sky.darksky:(47.6062, -122.3321, 2019-12-12): Requesting from DarkSky with parameters {'query_parameters': {'exclude': ['alerts', 'flags', 'hourly', 'minutely', 'offset', 'currently']}}


{'latitude': 47.6062,
 'longitude': -122.3321,
 'queried_date_utc': '2019-12-12',
 'weather_date_local': '2019-11-10',
 'weather_json': '{"cloud_cover_pct": 0.93, "dew_point_f": 48.57, "humidity_pct": 0.94, "icon": "fog", "moon_phase_pct": 0.46, "ozone": 262.2, "precip_intensity_avg_in_hr": 0.0004, "precip_intensity_max_in_hr": 0.0024, "precip_probability": 0.34, "precip_type": "rain", "pressure": 1026.1, "summary": "Foggy in the morning.", "sunrise_time_local": "2019-11-10T07:09:00-08:00", "sunset_time_local": "2019-11-10T16:41:00-08:00", "temperature_max_f": 53.25, "temperature_min_f": 44.73, "uv_index": 1, "visibility_mi": 5.174, "weather_date_local": "2019-11-10", "wind_gust_mph": 11.25, "wind_speed_mph": 3.93}'}

#### Serialize the output

```
{
    "0": {
        "latitude": 42.3601,
        "longitude": -71.0589,
        "queried_date_utc": "2019-11-10",
        "weather_date_local": "2019-11-10",
        "weather_json": weather_json,
    },
    "1": {
        "latitude": 42.3601,
        "longitude": -71.0589,
        "queried_date_utc": "2019-01-01",
        "weather_date_local": "2019-11-11",
        "weather_json": weather_json,
    },
    ...
}
```

In [5]:
serialized_daily_weather_forecast = darksky._serialize(response=darksky_weather_response)

print(serialized_daily_weather_forecast.keys())
serialized_daily_weather_forecast["daily_forecast"][0]

INFO:rogue_sky.darksky:(47.6062, -122.3321, 2019-12-12): Serializing to API output...


dict_keys(['latitude', 'longitude', 'queried_date_utc', 'daily_forecast'])


{'cloud_cover_pct': 0.93,
 'dew_point_f': 48.57,
 'humidity_pct': 0.94,
 'icon': 'fog',
 'moon_phase_pct': 0.46,
 'ozone': 262.2,
 'precip_intensity_avg_in_hr': 0.0004,
 'precip_intensity_max_in_hr': 0.0024,
 'precip_probability': 0.34,
 'precip_type': 'rain',
 'pressure': 1026.1,
 'summary': 'Foggy in the morning.',
 'sunrise_time_local': '2019-11-10T07:09:00-08:00',
 'sunset_time_local': '2019-11-10T16:41:00-08:00',
 'temperature_max_f': 53.25,
 'temperature_min_f': 44.73,
 'uv_index': 1,
 'visibility_mi': 5.174,
 'weather_date_local': '2019-11-10',
 'wind_gust_mph': 11.25,
 'wind_speed_mph': 3.93}

### Understanding the data

```
{
    'latitude': 47.6062,
    'longitude': -122.3321,
    'timezone': 'America/Los_Angeles',
    'currently': dict  current weather,
    'minutely': {
        'icon': 'cloudy',
        'data': [dict]  forecast by minute for 60 minutes
    },
    'hourly': {
        'summary': 'cloudy',
        'icon': 'cloudy',
        'data': [dict]  forecast by hour for 48 hours
    },
    'daily': {
        'summary': 'cloudy',
        'icon': 'cloudy',
        'data': [dict]  forecast by day for 8 days, including today
    }
    
```

In [6]:
# the response from DarkSky

with open("resources/darksky_response.json") as json_file:
    darksky_api_data = json.load(json_file)

In [7]:
darksky_api_data.keys()

dict_keys(['latitude', 'longitude', 'timezone', 'currently', 'minutely', 'hourly', 'daily', 'flags', 'offset'])

In [8]:
darksky_api_data["daily"].keys()

dict_keys(['summary', 'icon', 'data'])

In [9]:
type(darksky_api_data["daily"]["data"])

list

In [10]:
list(darksky_api_data["daily"]["data"][0].keys())

['time',
 'summary',
 'icon',
 'sunriseTime',
 'sunsetTime',
 'moonPhase',
 'precipIntensity',
 'precipIntensityMax',
 'precipIntensityMaxTime',
 'precipProbability',
 'precipType',
 'temperatureHigh',
 'temperatureHighTime',
 'temperatureLow',
 'temperatureLowTime',
 'apparentTemperatureHigh',
 'apparentTemperatureHighTime',
 'apparentTemperatureLow',
 'apparentTemperatureLowTime',
 'dewPoint',
 'humidity',
 'pressure',
 'windSpeed',
 'windGust',
 'windGustTime',
 'windBearing',
 'cloudCover',
 'uvIndex',
 'uvIndexTime',
 'visibility',
 'ozone',
 'temperatureMin',
 'temperatureMinTime',
 'temperatureMax',
 'temperatureMaxTime',
 'apparentTemperatureMin',
 'apparentTemperatureMinTime',
 'apparentTemperatureMax',
 'apparentTemperatureMaxTime']

In [11]:
# times are in UNIX time
# `time` is in local time, but all other times are in UTC (e.g. `sunriseTime`)

darksky_api_data["daily"]["data"][0]

{'time': 1573372800,
 'summary': 'Foggy in the morning.',
 'icon': 'fog',
 'sunriseTime': 1573398540,
 'sunsetTime': 1573432860,
 'moonPhase': 0.46,
 'precipIntensity': 0.0004,
 'precipIntensityMax': 0.0024,
 'precipIntensityMaxTime': 1573448400,
 'precipProbability': 0.34,
 'precipType': 'rain',
 'temperatureHigh': 53.25,
 'temperatureHighTime': 1573422960,
 'temperatureLow': 40.82,
 'temperatureLowTime': 1573481400,
 'apparentTemperatureHigh': 52.75,
 'apparentTemperatureHighTime': 1573422960,
 'apparentTemperatureLow': 38.74,
 'apparentTemperatureLowTime': 1573481520,
 'dewPoint': 48.57,
 'humidity': 0.94,
 'pressure': 1026.1,
 'windSpeed': 3.93,
 'windGust': 11.25,
 'windGustTime': 1573434060,
 'windBearing': 353,
 'cloudCover': 0.93,
 'uvIndex': 1,
 'uvIndexTime': 1573415700,
 'visibility': 5.174,
 'ozone': 262.2,
 'temperatureMin': 44.73,
 'temperatureMinTime': 1573459200,
 'temperatureMax': 53.25,
 'temperatureMaxTime': 1573422960,
 'apparentTemperatureMin': 42.78,
 'apparentTem

## Star Predictions

### Using the stars main entry point...

Let's see it all work together... The main entry point into the stars module is the `get_star_forecast` method. This method:
    1. Hits the DarkSky API for the daily weather forecast and predicts the daily star visibility forecast from the daily weather forecast
    3. Serializes the daily star visibility forecast into a JSON-parseable response that can be sent from the RogueSky API to the RogueSky frontend00

The final output is of this form: 
```
{
    "0": {
        "latitude": 42.3601,
        "longitude": -71.0589,
        "queried_date_utc": "2019-01-01",
        "weather_date_local": "2019-11-10",
        "prediction": 0.7,
    },
    "1": {
        "latitude": 42.3601,
        "longitude": -71.0589,
        "queried_date_utc": "2019-01-01",
        "weather_date_local": "2019-11-11",
        "prediction": 0.1,
    },
    ...
}
```

where 0 is no visibility, and 1 is perfect visibility.

In [12]:
with mock_darksky_api():
    backend_star_response = stars.get_star_forecast(
        latitude=COORDINATES[0], 
        longitude=COORDINATES[1],
        api_key=DARKSKY_API_KEY,
    )

print(backend_star_response.keys())
print(backend_star_response["daily_forecast"][0])

INFO:rogue_sky.stars:Getting daily star visibility forecast for (47.6062, -122.3321) on 2019-12-12
INFO:rogue_sky.darksky:Getting daily weather forecast for (47.6062, -122.3321) on 2019-12-12
INFO:rogue_sky.darksky:(47.6062, -122.3321, 2019-12-12): Requesting from DarkSky with parameters {'query_parameters': {'exclude': ['alerts', 'flags', 'hourly', 'minutely', 'offset', 'currently']}}
INFO:rogue_sky.darksky:(47.6062, -122.3321, 2019-12-12): Serializing to API output...
INFO:rogue_sky.stars:(47.6062, -122.3321, 2019-12-12): Serializing to API output...


dict_keys(['latitude', 'longitude', 'queried_date_utc', 'daily_forecast', 'city', 'state'])
{'cloud_cover_pct': 0.93, 'dew_point_f': 48.57, 'humidity_pct': 0.94, 'icon': 'fog', 'moon_phase_pct': 0.46, 'ozone': 262.2, 'precip_intensity_avg_in_hr': 0.0004, 'precip_intensity_max_in_hr': 0.0024, 'precip_probability': 0.34, 'precip_type': 'rain', 'pressure': 1026.1, 'summary': 'Foggy in the morning.', 'sunrise_time_local': '2019-11-10T07:09:00-08:00', 'sunset_time_local': '2019-11-10T16:41:00-08:00', 'temperature_max_f': 53.25, 'temperature_min_f': 44.73, 'uv_index': 1, 'visibility_mi': 5.174, 'weather_date_local': '2019-11-10', 'wind_gust_mph': 11.25, 'wind_speed_mph': 3.93, 'star_visibility': 0.07}
